In [1]:
import os
from superduperdb import superduper
from superduperdb.backends.mongodb import Collection

mongodb_uri = os.getenv("MONGODB_URI", "mongodb://localhost:27017/volvo")

# SuperDuperDB, now handles your MongoDB database
# It just super dupers your database 
db = superduper(mongodb_uri, artifact_store='filesystem://data/artifacts/', downloads_folder='./data/downloads')

/home/ubuntu/project/pocs/volvo/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-16 08:37:16,461	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


 2024-Jan-16 08:37:16.46| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.base.build:60   | Data Client is ready. MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000)
 2024-Jan-16 08:37:16.47| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.base.build:35   | Connecting to Metadata Client with engine:  MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000)
 2024-Jan-16 08:37:16.47| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.base.datalayer:80   | Building Data Layer


## Load Data

In [2]:
from superduperdb.ext.unstructured.encoder import unstructured_encoder
from superduperdb import Document

TEST_FILE = "volvo.pdf"
db.add(unstructured_encoder)
collection = Collection('source')
to_insert = [Document({"elements": unstructured_encoder(TEST_FILE)})]
db.execute(collection.insert_many(to_insert))

[2024-01-16 08:37:20] pikepdf._core INFO pikepdf C++ to Python logger bridge initialized


 2024-Jan-16 08:38:19.78| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.backends.local.compute:32   | Submitting job. function:<function callable_job at 0x7fe1f8134040>
 2024-Jan-16 08:38:20.03| SUCCESS  | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.backends.local.compute:38   | Job submitted.  function:<function callable_job at 0x7fe1f8134040> future:ec1cfd52-754f-4880-b373-d6dd8595e2a2


([ObjectId('65a6407b8b8f084ae65a2e69')],
 TaskWorkflow(database=<superduperdb.base.datalayer.Datalayer object at 0x7fe1a1498880>, G=<networkx.classes.digraph.DiGraph object at 0x7fe0d513a5c0>))

## Chunk Model

In [3]:
from collections import defaultdict
from superduperdb import Listener, Model, Schema, vector

STRIDE = 3  # stride in numbers of lines
WINDOW = 10  # length of window in numbers of lines
MAX_NUM = 999999999

def merge_metadatas(metadatas):

    if not metadatas:
        return {}
    p1, p2, p3, p4 = (MAX_NUM, MAX_NUM), (MAX_NUM, 0), (0, 0), (0, MAX_NUM)
    for metadata in metadatas:
        p1_, p2_, p3_, p4_ = metadata['coordinates']['points']
        p1 = (min(p1[0], p1_[0]), min(p1[1], p1_[1]))
        p2 = (min(p2[0], p2_[0]), max(p2[1], p2_[1]))
        p3 = (max(p3[0], p3_[0]), max(p3[1], p3_[1]))
        p4 = (max(p4[0], p4_[0]), min(p4[1], p4_[1]))
    points = (p1, p2, p3, p4)
    page_number = metadata['page_number']
    return {'points': points, 'page_number': page_number}

def create_chunk_and_metadatas(page_elements):
    datas = []
    for i in range(0, len(page_elements), STRIDE):
        windown_elements = page_elements[i : i + WINDOW]
        metadatas = [e.metadata.to_dict() for e in windown_elements]
        chunk = "\n".join([e.text for e in windown_elements])
        datas.append({"txt": chunk, 'metadata': merge_metadatas(metadatas)})
    return datas

def get_chunks(elements):
    from collections import defaultdict

    pages_elements = defaultdict(list)
    for element in elements:
        pages_elements[element.metadata.page_number].append(element)

    all_chunks_and_links = sum(
        [
            create_chunk_and_metadatas(page_elements)
            for _, page_elements in pages_elements.items()
        ],
        [],
    )
    return all_chunks_and_links

chunk_model = Model(
    identifier="chunk",
    object=get_chunks,
    flatten=True,
    model_update_kwargs={"document_embedded": False},
    output_schema=Schema(identifier="myschema", fields={"txt": "string"}),
)



db.add(
    Listener(
        model=chunk_model,  # Assuming video2images is your SuperDuperDB model
        select=collection.find(),
        key="elements",
    )
)

1it [00:00,  2.56it/s]


 2024-Jan-16 08:38:22.93| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.components.model:477  | Adding 1 model outputs to `db`


([None],
 Listener(identifier='chunk/elements', key='elements', model=Model(identifier='chunk', encoder=None, output_schema=Schema(identifier='myschema', fields={'txt': 'string', '_fold': FieldType(identifier='String')}), flatten=True, preprocess=None, postprocess=None, collate_fn=None, batch_predict=False, takes_context=False, metrics=(), model_update_kwargs={'document_embedded': False}, validation_sets=None, predict_X=None, predict_select=None, predict_max_chunk_size=None, predict_kwargs=None, object=<Artifact artifact=<function get_chunks at 0x7fe1827ad870> serializer=dill>, model_to_device_method=None, metric_values={}, predict_method=None, serializer='dill', device='cpu', preferred_devices=('cuda', 'mps', 'cpu'), training_configuration=None, train_X=None, train_y=None, train_select=None), select=<superduperdb.backends.mongodb.query.MongoCompoundSelect[
     source.find({'_id': "{'$in': '[65a6407b8b8f084ae65a2e69]'}"}, {})}
 ] object at 0x7fe0ce3caf20>, active=True, predict_kwargs=

## Create a vector-search Index

In [4]:
import sentence_transformers
from superduperdb import vector, VectorIndex
chunk_collection = Collection("_outputs.elements.chunk")

model = Model(
    identifier="embedding",
    object=sentence_transformers.SentenceTransformer("BAAI/bge-large-en-v1.5"),
    encoder=vector(shape=(384,)),
    predict_method="encode",  # Specify the prediction method
    preprocess=lambda x:x['0']['txt'] if isinstance(x, dict) else x,
    postprocess=lambda x: x.tolist(),  # Define postprocessing function
    batch_predict=True,  # Generate predictions for a set of observations all at once
)

db.add(
    VectorIndex(
        # Use a dynamic identifier based on the model's identifier
        identifier="vector-index",
        # Specify an indexing listener with MongoDB collection and model
        indexing_listener=Listener(
            select=chunk_collection.find(),
            key="_outputs.elements.chunk",  # Key for the documents
            model=model,  # Specify the model for processing
            predict_kwargs={"max_chunk_size": 1000},
        ),
    )
)

[2024-01-16 08:38:23] sentence_transformers.SentenceTransformer INFO Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5
[2024-01-16 08:38:23] sentence_transformers.SentenceTransformer INFO Use pytorch device: cuda
3661it [00:00, 28933.70it/s]


 2024-Jan-16 08:38:28.02| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.components.model:417  | Computing chunk 0/3


Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:07<00:00,  4.09it/s]


 2024-Jan-16 08:38:36.29| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.components.model:477  | Adding 1000 model outputs to `db`
 2024-Jan-16 08:38:36.59| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.components.model:417  | Computing chunk 1/3


Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  6.05it/s]


...

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:04<00:00,  6.54it/s]


 2024-Jan-16 08:38:47.54| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.components.model:477  | Adding 1000 model outputs to `db`
 2024-Jan-16 08:38:47.81| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.components.model:417  | Computing chunk 3/3


Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:05<00:00,  3.57it/s]


 2024-Jan-16 08:38:53.79| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.components.model:477  | Adding 661 model outputs to `db`


([None],
 VectorIndex(identifier='vector-index', indexing_listener=Listener(identifier='embedding/elements', key='_outputs.elements.chunk', model=Model(identifier='embedding', encoder=Encoder(identifier='vector[384]', decoder=None, encoder=None, shape=(384,), load_hybrid=True), output_schema=None, flatten=False, preprocess=<Artifact artifact=<function <lambda> at 0x7fe0d15e7640> serializer=dill>, postprocess=<Artifact artifact=<function <lambda> at 0x7fe0ce54bf40> serializer=dill>, collate_fn=None, batch_predict=True, takes_context=False, metrics=(), model_update_kwargs={}, validation_sets=None, predict_X=None, predict_select=None, predict_max_chunk_size=None, predict_kwargs=None, object=<Artifact artifact=SentenceTransformer(
   (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
   (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling

## Search Content from PDF

In [5]:
query = "How to call support"
out = db.execute(
    chunk_collection.like(
        Document({"_outputs.elements.chunk": query}),
        vector_index="vector-index",
        n=5,
    ).find({})
)

for r in sorted(out, key=lambda x:x.content['score'], reverse=True):
    print(r.content['score'])
    x = r.outputs('elements', 'chunk')['txt']
    print(x)

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 44.40it/s]


 2024-Jan-16 08:38:54.06| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.base.datalayer:132  | loading of vectors of vector-index: 'vector-index'
 2024-Jan-16 08:38:54.06| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.base.datalayer:166  | <superduperdb.backends.mongodb.query.MongoCompoundSelect[
    _outputs.elements.chunk.elements.chunk.find({'_id': "{'$in': '[65a6407e8b8f084ae65a3a27, 65a6407e8b8f084ae65a3a28, 65a6407e8b8f084ae65a3a29, 65a6407e8b8f084ae65a3a2a, 65a6407e8b8f084ae65a3a2b, 65a6407e8b8f084ae65a3a2c, 65a6407e8b8f084ae65a3a2d, 65a6407e8b8f084ae65a3a2e, 65a6407e8b8f084ae65a3a2f, 65a6407e8b8f084ae65a3a30, 65a6407e8b8f084ae65a3a31, 65a6407e8b8f084ae65a3a32, 65a6407e8b8f084ae65a3a33, 65a6407e8b8f084ae65a3a34, 65a6407e8b8f084ae65a3a35, 65a6407e8b8f084ae65a3a36, 65a6407e8b8f084ae65a3a37, 65a6407e8b8f084ae65a3a38, 65a6407e8b8f084ae65a3a39, 65a6407e8b8f084ae65a3a3a, 65a6407e8b8f084ae65a3a3b, 65a6407e8b8f084ae65a3a

Loading vectors into vector-table...: 661it [00:00, 897.34it/s]

0.6128491271978982
Warranty
6 1
y t n a r r a W
389
0.595966817797636
Contact Volvo Action Service, they have the equipment and knowledge to help you.
AA 338361
0.5908166713587972
1 Remove the negative cable.
2 Remove the positive cable.
Emergency starting
4 1
N O
I T C A
Y C N E G R E M E
363
0.5877470887268926
NOTE
Make sure the earth connection is clean and free from paint.
Remove charge cables
1 Remove the negative cable.
2 Remove the positive cable.
Emergency starting
4 1
N O
I T C A
Y C N E G R E M E
0.5782561791891998
EMERGENCY ACTION
14


## QA System For PDF

In [6]:
from superduperdb.ext.llm.vllm import VllmModel

# Define the prompt for the llm model
prompt_template = (
    'The following is a document and question about the volvo user manual\n'
    'Only provide a very concise answer\n'
    '{context}\n\n'
    'Here\'s the question:{input}\n'
    'answer:'
)

# Create an instance of llm with the specified model and prompt
llm = VllmModel(identifier='llm',
                 model_name='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', 
                 prompt_template=prompt_template,
                 vllm_kwargs={"max_model_len": 2048, "quantization": "awq"}, 
                 inference_kwargs={"max_tokens":2048},
               )

# Add the llm instance
db.add(llm)

# Print information about the models in the SuperDuperDB database
print(db.show('model'))

['chunk', 'embedding', 'llm']


In [7]:
from superduperdb import Document

# Use the SuperDuperDB model to generate a response based on the search term and context
output, out = db.predict(
    model_name='llm',
    input=query,
    context_select=chunk_collection.like(
        Document({"_outputs.elements.chunk": query}),
        vector_index="vector-index",
        n=5,
    ).find({}),
    context_key='_outputs.elements.chunk.0.txt',
)

# Get the reference links corresponding to the answer context
datas = []
page_messages = []
for source in sorted(out, key=lambda x:x.content['score'], reverse=True):
    chunk_data = source.outputs('elements', 'chunk')
    metadata = chunk_data['metadata']
    page_number = metadata['page_number']
    points = metadata['points']
    score = source["score"]
    message = f"page_number: {page_number}, score: {score:3f}, coordinates: {points}"
    page_messages.append(message)
    

# Display the generated response using Markdown
print(output.content)
print("\n".join(page_messages))

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 54.33it/s]

 2024-Jan-16 08:38:55.03| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.ext.llm.base:29   | Initializing VllmModel : llm
WARNING 01-16 08:38:55 config.py:175] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 01-16 08:38:55 llm_engine.py:70] Initializing an LLM engine with config: model='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', tokenizer='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=awq, enforce_eager=False, seed=0)


INFO 01-16 08:38:58 llm_engine.py:275] # GPU blocks: 6692, # CPU blocks: 2048
INFO 01-16 08:39:00 model_runner.py:501] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-16 08:39:00 model_runner.py:505] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode.
INFO 01-16 08:39:11 model_runner.py:547] Graph capturing finished in 11 secs.
 2024-Jan-16 08:39:11.12| INFO     | ip-172-31-16-121| 9eb708c6-a9d9-4c83-ab08-80146246024d| superduperdb.ext.llm.base:32   | Initialized  VllmModel : llm successfully
 Contact Volvo Action Service. The number is AA 338361.
page_number: 399, score: 0.612849, coordinates: [[483.11682, 16.42147], [483.11682, 397.54553999999996], [579.45278, 397.54553999999996], [579.45278, 16.42147]]
page_number: 374,